In [ ]:
import sys,os
import pandas as pd
#https://www.sbert.net/examples/applications/semantic-search/README.html
sys.path.append("../input/sentence-bert")
from sentence_transformers import SentenceTransformer,util

In [ ]:
model = SentenceTransformer("../input/alldistilrobertav1")
df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
ruddiit = pd.read_csv("/kaggle/input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv")
ruddiit['offensiveness_score'] = (ruddiit['offensiveness_score'] - ruddiit['offensiveness_score'].min()) / (ruddiit['offensiveness_score'].max() - ruddiit['offensiveness_score'].min()) 
ru_text = ruddiit["txt"].values
ru_toxic_score = ruddiit["offensiveness_score"].values

jig_text = df['text'].values

In [ ]:

df_jig1 = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
#df_jig1['severe_toxic'] = df_jig1.severe_toxic * 3
#df_jig1['y'] = (df_jig1[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) ).astype(int)

cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    df_jig1[category] = df_jig1[category] * cat_mtpl[category]

df_jig1['y'] = df_jig1.loc[:, 'toxic':'identity_hate'].sum(axis=1)
ru_text = df_jig1["comment_text"].values
ru_toxic_score = df_jig1["y"].values#0.6513551215623754 75 170s

In [ ]:
corpus_embeddings = model.encode(ru_text, convert_to_tensor=True)
query_embeddings = model.encode(jig_text, convert_to_tensor=True)
K=75
corpus_embeddings = corpus_embeddings.to('cuda')
corpus_embeddings = util.normalize_embeddings(corpus_embeddings)

query_embeddings = query_embeddings.to('cuda')
query_embeddings = util.normalize_embeddings(query_embeddings)

hits = util.semantic_search(query_embeddings, corpus_embeddings, score_function=util.dot_score,top_k=K)

In [ ]:
import numpy as np
def check(idx):
    c = 0
    d = 0
    for i in range(K):
        #top1_txt = ru_text[hits[0][i]["corpus_id"]]
        more_score = ru_toxic_score[hits[idx][i]["corpus_id"]]
        #print(hits[idx][i])
        #more_w = hits[idx][i]["score"]
        #less_score = ru_toxic_score[hits_less[idx][i]["corpus_id"]]
        #less_w = hits_less[idx][i]["score"]

        c+=more_score#*more_w
    return c/K

sentence_bert_sore = np.empty(len(df))

for i in range(len(df)):
    sentence_bert_sore[i]=check(i)

In [ ]:

df["score"]=sentence_bert_sore
df['score'] = df['score'].rank(method='first')
df[['comment_id', 'score']].to_csv("submission.csv", index=False)

In [ ]:
df